# Predicting Biological Response
* Beohringer Ingelheim 의 Kaggle project <br> https://www.kaggle.com/c/bioresponse
* 특정 성분(molecule)의 생물반응(Biological Response)를 예측하는 프로젝트
* logloss가 낮은 모델을 만드는 것이 목표

In [1031]:
import pandas as pd # Analysis
import numpy as np # Analysis
from matplotlib import pyplot as plt # Visualize
from xgboost import XGBClassifier # Classification model
from lightgbm import LGBMClassifier # Classification model
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier # Classification model
from sklearn.svm import SVC # Classification model
from sklearn.decomposition import PCA # Preprocessing
from sklearn.preprocessing import StandardScaler # Preprocessing
from sklearn.metrics import log_loss # Validation
from sklearn.model_selection import GridSearchCV # Validation
%matplotlib inline

# Data Import

In [1032]:
df = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [1033]:
df.tail()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
3746,1,0.033300,0.506409,0.10,0.0,0.209887,0.633426,0.297659,0.376124,0.727093,...,0,0,0,0,0,0,0,0,0,0
3747,1,0.133333,0.651023,0.15,0.0,0.151154,0.766505,0.170876,0.404546,0.787935,...,0,0,1,0,1,0,1,0,0,0
3748,0,0.200000,0.520564,0.00,0.0,0.179949,0.768785,0.177341,0.471179,0.872241,...,0,0,0,0,0,0,0,0,0,0
3749,1,0.100000,0.765646,0.00,0.0,0.536954,0.634936,0.342713,0.447162,0.672689,...,0,0,0,0,0,0,0,0,0,0
3750,0,0.133333,0.533952,0.00,0.0,0.347966,0.757971,0.230667,0.272652,0.854116,...,0,0,0,0,0,0,0,0,0,0


In [1034]:
test.tail()

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
2496,0.066700,0.658812,0.10,0.0,0.305799,0.614877,0.180900,0.219328,0.617916,0.324679,...,0,0,0,0,0,0,0,0,0,0
2497,0.033300,0.451048,0.00,0.0,0.230019,0.849600,0.114983,0.159589,0.916702,0.043200,...,0,0,0,0,0,0,0,0,0,0
2498,0.000000,0.537887,0.15,0.0,0.144312,0.667734,0.283773,0.591918,0.760417,0.275136,...,0,0,0,0,0,0,0,0,0,0
2499,0.033300,0.538504,0.10,0.0,0.191739,0.577244,0.305091,0.554121,0.676559,0.385720,...,0,0,0,0,0,0,0,0,0,0
2500,0.166667,0.648932,0.05,0.0,0.225382,0.619299,0.329329,0.522098,0.704095,0.339546,...,0,0,0,0,0,0,0,0,0,0


* 각 Data Set의 행은 개별 성분을 의미하고, 열은 성분의 특성을 의미한다.
* 열을 구성하는 특성에는 분자의 크기와 모양, 구성요소 등이 포함되나, 이름이 D1~D1776으로 대체되었고 그 값도 정규화 되어있다.
* 변수값은 연속형과 이산형이 혼합되어있다.
* Train Set에는 생물반응 여부가 Activity 열에 표기되어 있다. (없음: 0, 있음: 1)
* Train Set은 3750개의 행으로, Test Set은 2500개의 행으로 구성되어 있다.

In [1035]:
df['Activity'].value_counts()

1    2034
0    1717
Name: Activity, dtype: int64

독립변수는 df_X로, 종속변수는 df_y로 분리

In [1036]:
df_X = df.iloc[:, 1:]

In [1037]:
df_X.head()

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,0.243144,...,0,0,0,0,0,0,0,0,0,0
1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,0.106480,...,1,1,1,1,0,1,0,0,1,0
2,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,0.352308,...,0,0,0,0,0,0,0,0,0,0
3,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,0.208989,...,0,0,0,0,0,0,0,0,0,0
4,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,0.125177,...,0,0,0,0,0,0,0,0,0,0


In [1038]:
df_y = df.iloc[:, :1]
df_y.head()

,Activity
0,1
1,1
2,1
3,1
4,0


* row는 3750개로 많지 않으나, column이 1776개로 많다.
* 변수가 너무 많은 것으로 판단되어 PCA를 적용한 데이터셋을 생성 한다.

In [1039]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(df_X, df_y, random_state=0)

In [1040]:
train_y = train_y.values.ravel()
test_y = test_y.values.ravel()

# PCA

In [1041]:
pca = PCA()
scaler = StandardScaler()

In [1042]:
pca.fit(df_X)

PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [1043]:
opt_n = pca.explained_variance_ratio_

In [1044]:
# 설명력이 90%가 되는 시점의 components 갯수를 출력
def optimal_n(start, ratio, x):
    n = start
    def sum_var(n, ratio, x, comp):
        ratio += comp
        if ratio >= 0.90:
            return '90%: {}, {}'.format(n, ratio)
        else:
            return optimal_n(n, ratio, x)
    comp = x[n]
    n += 1
    return sum_var(n, ratio, x, comp)

In [1045]:
optimal_n(0, 0, opt_n)

'90%: 242, 0.9001024831518873'

* PCA fit결과, 242개 차원으로 90%의 설명력을 가질 수 있는 것으로 판단된다.
* 상기 결과는 전체 데이터 셋을 대상으로 하였으므로, 200개로 PCA를 수행.

In [1046]:
pca = PCA(n_components=200)
pca.fit(train_X)

PCA(copy=True, iterated_power='auto', n_components=200, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [1047]:
pca_train = pca.transform(train_X)
pca_test = pca.transform(test_X)
pca_train = scaler.fit_transform(pca_train)
pca_test = scaler.transform(pca_test)

In [1048]:
pca_train.shape

(2813, 200)

In [1049]:
train_X.shape

(2813, 1776)

# Model selection
* 모델마다 train set, PCA set을 교차하여 검증

In [1050]:
# Cross Validation
def model_cv(train, test, train_y, test_y, model, name):
    model.fit(train, train_y)
    print(name,': ',model.best_params_)
    pred_y = model.predict(test)
    print('train score: {}'.format(model.score(train, train_y)))
    print('test score: {}'.format(model.score(test, test_y)))
    print('log loss: {}'.format(log_loss(test_y, pred_y)))
    print()

In [1051]:
# Models 
def forest(train, test, train_y, test_y):
    param = [{'n_estimators':[100, 300, 500, 1000],
         'criterion': ['gini', 'entropy']
         }]
    model = GridSearchCV(RandomForestClassifier(n_jobs=-1), param, cv=3)
    name = 'Random forest'
    return model_cv(train, test, train_y, test_y, model, name)
    
def grbc(train, test, train_y, test_y):
    param = [{'n_estimators': [100, 500, 1000],
         'learning_rate': [0.1, 0.05, 0.01]}]
    name = 'Gradient boosting'
    model = GridSearchCV(GradientBoostingClassifier(), param, cv=3)
    return model_cv(train, test, train_y, test_y, model, name)
    
def xgbc(train, test, train_y, test_y):
    param = [{'n_estimators': [100, 500, 1000],
         'learning_rate': [0.1, 0.05, 0.01],}]
    model = GridSearchCV(XGBClassifier(), param)
    name = 'XGBoost'
    return model_cv(train, test, train_y, test_y, model, name)

def lgbm(train, test, train_y, test_y):
    param = [{'n_estimators': [100, 500, 1000],
         'learning_rate': [0.1, 0.05, 0.01]}]
    model = GridSearchCV(LGBMClassifier(), param)
    name = 'LightGBM'
    return model_cv(train, test, train_y, test_y, model, name)

In [1052]:
# Random forest

print('Train set')
forest(train_X, test_X , train_y, test_y)
print('PCA set')
forest(pca_train, pca_test, train_y, test_y)

Train set
Random forest :  {'criterion': 'gini', 'n_estimators': 500}
train score: 1.0
test score: 0.7963752665245203
log loss: 7.033030120665042

PCA set
Random forest :  {'criterion': 'entropy', 'n_estimators': 1000}
train score: 1.0
test score: 0.7579957356076759
log loss: 8.358642845317783



In [503]:
# LightGBM

print('Train set')
lgbm(train_X, test_X , train_y, test_y)
print('PCA set')
lgbm(pca_train, pca_test, train_y, test_y)

Train set
LightGBM :  {'learning_rate': 0.01, 'n_estimators': 1000}
train score: 0.9971560611446854
test score: 0.8166311300639659
log loss: 6.333413969393161

PCA set
LightGBM :  {'learning_rate': 0.01, 'n_estimators': 1000}
train score: 0.9992890152861713
test score: 0.779317697228145
log loss: 7.622198151216103



In [506]:
# XGBoost

print('Train set')
xgbc(train_X, test_X , train_y, test_y)
print('PCA set')
xgbc(pca_train, pca_test, train_y, test_y)

Train set
XGBoost :  {'learning_rate': 0.1, 'n_estimators': 500}
train score: 0.9765375044436545
test score: 0.8006396588486141
log loss: 6.885744932621564

PCA set
XGBoost :  {'learning_rate': 0.1, 'n_estimators': 1000}
train score: 1.0
test score: 0.7707889125799574
log loss: 7.916765117505916



In [512]:
# Gradient Boosting

print('Train set')
grbc(train_X, test_X , train_y, test_y)
print('PCA set')
grbc(pca_train, pca_test, train_y, test_y)

Train set
Gradient boosting :  {'learning_rate': 0.1, 'n_estimators': 500}
train score: 0.9790259509420548
test score: 0.7953091684434968
log loss: 7.069858663494841

PCA set
Gradient boosting :  {'learning_rate': 0.01, 'n_estimators': 1000}
train score: 0.9057945254177036
test score: 0.7643923240938166
log loss: 8.137711653455423



# PCA is not an answer
* PCA를 적용한 set의 결과가 더 좋지 않았음.
* 각 변수들이 중복되지 않고 각각의 역할을 한다고 보임.
* 이에 새로운 변수를 추가하는 방안을 고려.
* 4개 모델의 predict proba를 새로운 변수로 추가.

In [978]:
forest = RandomForestClassifier(criterion='entropy', n_estimators=1000, n_jobs=-1)
lgbm = LGBMClassifier(learning_rate=0.01, n_estimators=1000)
xgbc = XGBClassifier(learning_rate=0.1, n_estimators=500)
grbc = GradientBoostingClassifier(learning_rate=0.1, n_estimators=500)
models = [forest, lgbm, xgbc, grbc]

In [979]:
forest.fit(train_X, train_y)
lgbm.fit(train_X, train_y)
xgbc.fit(train_X, train_y)
grbc.fit(train_X, train_y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=500, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)

In [980]:
add = pd.DataFrame({0:[]})

train_forest = forest.predict_proba(train_X)
train_lgbm = lgbm.predict_proba(train_X)
train_xgbc = xgbc.predict_proba(train_X)
train_grbc = grbc.predict_proba(train_X)

In [981]:
tf_X = train_X.copy()
tf_X[0] = pd.DataFrame(train_forest[:,1])
tf_X[1] = pd.DataFrame(train_lgbm[:,1])
tf_X[2] = pd.DataFrame(train_xgbc[:,1])
tf_X[3] = pd.DataFrame(train_grbc[:,1])

In [982]:
tf_X.head()

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,...,D1771,D1772,D1773,D1774,D1775,D1776,0,1,2,3
704,0.133333,0.661264,0.0,0.00,0.290186,0.668006,0.278627,0.472782,0.750612,0.259261,...,0,0,0,0,0,0,0.123,0.030177,0.028683,0.032053
76,0.000000,0.597969,0.0,0.00,0.266245,0.624966,0.410535,0.604793,0.752670,0.365163,...,0,0,0,0,0,0,0.209,0.199541,0.228988,0.274487
3460,0.033300,0.597146,0.1,0.00,0.112010,0.627221,0.414013,0.610064,0.739563,0.379890,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
905,0.000000,0.660245,0.1,0.00,0.141600,0.611352,0.396943,0.612558,0.703160,0.364433,...,0,0,0,0,0,0,0.983,0.961682,0.971290,0.931981
9,0.100000,0.768150,0.1,0.25,0.262299,0.598972,0.377065,0.394514,0.673797,0.343472,...,0,0,0,0,0,0,0.843,0.878184,0.641850,0.686951


In [983]:
test_forest = forest.predict_proba(test_X)
test_lgbm = lgbm.predict_proba(test_X)
test_xgbc = xgbc.predict_proba(test_X)
test_grbc = grbc.predict_proba(test_X)

In [984]:
testf_X = test_X.copy()

In [985]:
testf_X[0] = pd.DataFrame(test_forest[:,1])
testf_X[1] = pd.DataFrame(test_lgbm[:,1])
testf_X[2] = pd.DataFrame(test_xgbc[:,1])
testf_X[3] = pd.DataFrame(test_grbc[:,1])

In [986]:
tf_X = tf_X.fillna(0)
testf_X = testf_X.fillna(0)

In [987]:
forest_ = RandomForestClassifier(criterion='entropy', n_estimators=1000, n_jobs=-1)
lgbm_ = LGBMClassifier(learning_rate=0.01, n_estimators=1000)
xgbc_ = XGBClassifier(learning_rate=0.1, n_estimators=500)
grbc_ = GradientBoostingClassifier(learning_rate=0.1, n_estimators=500)

In [1002]:
forest_.fit(tf_X, train_y)
print(forest_.score(tf_X, train_y))
print(forest_.score(testf_X, test_y))
pred_y = forest_.predict_proba(testf_X)
print('log loss: {}'.format(log_loss(test_y, pred_y)))

1.0
0.799573560768
log loss: 0.47331798578060846


In [1003]:
lgbm_.fit(tf_X, train_y)
print(lgbm_.score(tf_X, train_y))
print(lgbm_.score(testf_X, test_y))
pred_y = lgbm_.predict_proba(testf_X)
print('log loss: {}'.format(log_loss(test_y, pred_y)))

0.998222538215
0.794243070362
log loss: 0.47775719952956947


In [1004]:
xgbc_.fit(tf_X, train_y)
print(xgbc_.score(tf_X, train_y))
print(xgbc_.score(testf_X, test_y))
pred_y = xgbc_.predict_proba(testf_X)
print('log loss: {}'.format(log_loss(test_y, pred_y)))

0.983647351582
0.796375266525
log loss: 0.49151201889592544


In [1005]:
grbc_.fit(tf_X, train_y)
print(grbc_.score(tf_X, train_y))
print(grbc_.score(testf_X, test_y))
pred_y = grbc_.predict_proba(testf_X)
print('log loss: {}'.format(log_loss(test_y, pred_y)))

0.982225382154
0.796375266525
log loss: 0.48166088042389765


# Submit

In [1006]:
test = pd.read_csv('data/test.csv')

In [1007]:
test.tail()

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
2496,0.066700,0.658812,0.10,0.0,0.305799,0.614877,0.180900,0.219328,0.617916,0.324679,...,0,0,0,0,0,0,0,0,0,0
2497,0.033300,0.451048,0.00,0.0,0.230019,0.849600,0.114983,0.159589,0.916702,0.043200,...,0,0,0,0,0,0,0,0,0,0
2498,0.000000,0.537887,0.15,0.0,0.144312,0.667734,0.283773,0.591918,0.760417,0.275136,...,0,0,0,0,0,0,0,0,0,0
2499,0.033300,0.538504,0.10,0.0,0.191739,0.577244,0.305091,0.554121,0.676559,0.385720,...,0,0,0,0,0,0,0,0,0,0
2500,0.166667,0.648932,0.05,0.0,0.225382,0.619299,0.329329,0.522098,0.704095,0.339546,...,0,0,0,0,0,0,0,0,0,0


In [1008]:
y = np.zeros(len(test))

In [1009]:
def add_col(X, y, models, n):
    add = pd.DataFrame({n:[]})
    return prob(X, y, models, n, add)

def prob(X, y, models, n, add):
    if n < len(models):
        model = models[n]
        model.fit(X, y.values.ravel())
        proba = model.predict_proba(X)[:,1]
        add[n] = pd.Series(proba, index=X.index)
        n += 1
        return prob(X, y, models, n, add)
    X = pd.concat([X, add], axis=1)
    return X

In [1010]:
tf = test.copy()

In [1011]:
test_forest = forest.predict_proba(test)
test_lgbm = lgbm.predict_proba(test)
test_xgbc = xgbc.predict_proba(test)
test_grbc = grbc.predict_proba(test_X)

In [1012]:
tf[0] = pd.DataFrame(test_forest[:,1])
tf[1] = pd.DataFrame(test_lgbm[:,1])
tf[2] = pd.DataFrame(test_xgbc[:,1])
tf[3] = pd.DataFrame(test_grbc[:,1])

In [1013]:
tf = tf.fillna(0)

In [1014]:
tf.head()

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,...,D1771,D1772,D1773,D1774,D1775,D1776,0,1,2,3
0,0.366667,0.611765,0.05,0.0,0.110435,0.803973,0.106075,0.473965,0.835617,0.106452,...,0,1,0,0,1,0,0.816,0.892162,0.883500,0.735400
1,0.100000,0.758175,0.30,0.0,0.180128,0.621378,0.287144,0.503919,0.674919,0.403616,...,0,0,0,0,0,0,0.893,0.988865,0.990358,0.067280
2,0.100000,0.658812,0.10,0.0,0.243421,0.640959,0.312765,0.279784,0.686775,0.280301,...,0,0,0,0,0,0,0.586,0.450464,0.659976,0.062815
3,0.100000,0.655752,0.10,0.0,0.226978,0.776996,0.150657,0.336948,0.802121,0.125608,...,0,0,0,0,0,0,0.960,0.992442,0.983976,0.976448
4,0.000000,0.484851,0.00,0.0,0.561200,0.771463,0.244287,0.293096,0.717575,0.230842,...,0,0,0,0,0,0,0.280,0.013182,0.012059,0.980446


In [1025]:
pred = lgbm_.predict_proba(tf)

In [1026]:
submit = pd.DataFrame(pred, columns=lgbm_.classes_)

In [1027]:
submit.index += 1

In [1028]:
submit = submit.iloc[:,1]

In [1029]:
submit.head()

1    0.881515
2    0.990310
3    0.485185
4    0.990072
5    0.016068
Name: 1, dtype: float64

In [1030]:
submit.to_csv('data/submit_lgbm_.csv')